# Projet maison n° 3

In [13]:
# imports
import pandas as pd
import os
import numpy as np

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [65]:
# names us
def df_names_us():
    
    filelist = [f for f in os.listdir('./names') if f.endswith('.txt')]
    filelist.sort()
    
    #filelist = ['yob1880.txt','yob1881.txt','yob1882.txt','yob1883.txt']
    li = []
    df = pd.DataFrame()
    
    for f in filelist:
        year = f[3:7]
        dftemp = pd.read_csv('./names/'+f,names=['name', 'gender', 'births'],dtype={'name':'str', 'gender':'str', 'births':'Int64'})
        dftemp.insert(0, 'year', year)
        df = df.append(dftemp,ignore_index=True)
        df['year'] = pd.to_numeric(df['year'], errors='coerce')

    return df

df_test = df_names_us()

display(df_test.head())
print(df_test.shape)
display(df_test.tail())
display(df_test.info())



,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


(2020863, 4)


,year,name,gender,births
2020858,2020,Zykell,M,5
2020859,2020,Zylus,M,5
2020860,2020,Zymari,M,5
2020861,2020,Zyn,M,5
2020862,2020,Zyran,M,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020863 entries, 0 to 2020862
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   year    int64 
 1   name    object
 2   gender  object
 3   births  Int64 
dtypes: Int64(1), int64(1), object(2)
memory usage: 63.6+ MB


None

In [66]:
df_test.info()
df_test.loc[df_test.isnull().any(axis=1)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020863 entries, 0 to 2020862
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   year    int64 
 1   name    object
 2   gender  object
 3   births  Int64 
dtypes: Int64(1), int64(1), object(2)
memory usage: 63.6+ MB


,year,name,gender,births


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [81]:
# names fr
def df_names_fr():
    
    naval = ['-1.#IND', '1.#QNAN', '1.#IND', '-1.#QNAN', '#N/A N/A', '#N/A', 'N/A', 'n/a', '<NA>', '#NA', 'NULL', 'null', 'NaN', '-NaN', 'nan', '-nan', '']
    
    df = pd.read_csv('nat2020.csv', sep=';', header=0, names=['gender','name','year','births'],dtype={'name':'str', 'gender':'str', 'births':'Int64', 'year':'str'},keep_default_na=False, na_values=naval)[['year','name', 'gender', 'births']]
    
    mapping = {'2': 'F', '1': 'M'}
    df['gender'] = df['gender'].map(mapping)
    
    df = df.drop(df[df.name.str.len() < 2].index)
    df = df.drop(df[df.name == '_PRENOMS_RARES'].index)
    
    df['name'] = df['name'].str.title()
    
    
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df = df.dropna(subset=['year'])
    df['year'] = df['year'].astype(int)
    
    df.sort_values(['year', 'gender', 'births', 'name'], ascending=[True, True, False, True], inplace=True, na_position='first')

    df = df.reset_index(drop=True)
    
    return df

df_french_test = df_names_fr()
display(df_french_test.head(10))
display(df_french_test.tail())
display(df_french_test.info())



,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
5,1900,Yvonne,F,5096
6,1900,Madeleine,F,4959
7,1900,Suzanne,F,4954
8,1900,Marthe,F,3771
9,1900,Marcelle,F,3766


,year,name,gender,births
630402,2020,Élouen,M,3
630403,2020,Énes,M,3
630404,2020,Éphraïm,M,3
630405,2020,Éros,M,3
630406,2020,Ézekiel,M,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630407 entries, 0 to 630406
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   year    630407 non-null  int32 
 1   name    630407 non-null  object
 2   gender  630407 non-null  object
 3   births  630407 non-null  Int64 
dtypes: Int64(1), int32(1), object(2)
memory usage: 17.4+ MB


None

In [28]:
df_french_test.info()
df_french_test['gender'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667364 entries, 0 to 667363
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   gender  667364 non-null  object
 1   name    667362 non-null  object
 2   year    667364 non-null  object
 3   births  667364 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 20.4+ MB


F    362931
M    304433
Name: gender, dtype: int64

In [82]:
# pd.to_numeric?

df_french_test[df_french_test.isna().any(axis=1)]

# df_french_test.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')]

,year,name,gender,births


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [ ]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
    
    
    
    return df

### Tests

In [83]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [84]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [85]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ERROR

ERROR: test_df_taux_change (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\schwa\AppData\Local\Temp/ipykernel_15844/3002108176.py", line 31, in test_df_taux_change
    df = df_taux_change(['CHF', 'GBP', 'USD'])
NameError: name 'df_taux_change' is not defined

----------------------------------------------------------------------
Ran 3 tests in 26.973s

FAILED (errors=1)
